# Converting NEXRAD Level 2 Data to Zarr Format

This notebook demonstrates the core workflow for converting NEXRAD Level 2 radar data into Analysis-Ready Cloud-Optimized (ARCO) Zarr format using the `raw2zarr` library.

```{note}
This example processes NEXRAD data from the AWS Open Data Registry and converts it to hierarchical Zarr format for efficient analysis.
```

## Import Libraries

In [1]:
import xarray as xr

from raw2zarr.builder.builder_utils import get_icechunk_repo

# Import the current API - convert_files is the main entry point
from raw2zarr.builder.convert import convert_files
from raw2zarr.utils import list_nexrad_files

## Discover NEXRAD Files

We'll use the `list_nexrad_files()` utility to find NEXRAD Level 2 radar files from the AWS Open Data Registry.

In [2]:
# Configuration
radar = "KVNX"  # Oklahoma radar station
zarr_store = f"../zarr/{radar}.zarr"  # Output Zarr store path

In [3]:
# Discover NEXRAD files for May 20, 2011 (sever weather outbreak day)
radar_files = list_nexrad_files(
    radar="KVNX", start_time="2011-05-20 9:00", end_time="2011-05-20 12:59"
)

print(f"Found {len(radar_files)} NEXRAD files for {radar}")

Found 54 NEXRAD files for KVNX


## Convert to Zarr Format

Convert radar files to Zarr format using sequential processing.

Create Icechunk repository for versioned Zarr storage.

In [4]:
# Create Icechunk repository using utility function
repo = get_icechunk_repo(zarr_store, use_manifest_config=True)

  2025-09-21T12:55:56.074204Z  WARN icechunk::storage::object_store: The LocalFileSystem storage is not safe for concurrent commits. If more than one thread/process will attempt to commit at the same time, prefer using object stores.
    at icechunk/src/storage/object_store.rs:80



In [5]:
%%time
convert_files(
    radar_files=radar_files,
    append_dim="vcp_time",
    repo=repo,
    zarr_format=3,
    engine="nexradlevel2",
    process_mode="sequential",
    remove_strings=True,
)

print(f"Conversion completed! Zarr store created at: {zarr_store}")

[icechunk] Committed s3://unidata-nexrad-level2/2011/05/20/KVNX/KVNX20110520_090333_V06.gz as snapshot 4YHDVN5VHFQYNGESYBGG
[icechunk] Committed s3://unidata-nexrad-level2/2011/05/20/KVNX/KVNX20110520_090753_V06.gz as snapshot GR1ZJPXVHDMN34G1XDV0
[icechunk] Committed s3://unidata-nexrad-level2/2011/05/20/KVNX/KVNX20110520_091212_V06.gz as snapshot QJF3K73T7NQ72JEM05ZG
[icechunk] Committed s3://unidata-nexrad-level2/2011/05/20/KVNX/KVNX20110520_091632_V06.gz as snapshot K9X616QSHE2NW2HYQHPG
[icechunk] Committed s3://unidata-nexrad-level2/2011/05/20/KVNX/KVNX20110520_092052_V06.gz as snapshot 39NKRQ7KAQWFSMG2GXF0
[icechunk] Committed s3://unidata-nexrad-level2/2011/05/20/KVNX/KVNX20110520_092709_V06.gz as snapshot A5V2CHDCZANQQRRZ7CZ0
[icechunk] Committed s3://unidata-nexrad-level2/2011/05/20/KVNX/KVNX20110520_093128_V06.gz as snapshot A918DXRVFD84DTD3RRVG
[icechunk] Committed s3://unidata-nexrad-level2/2011/05/20/KVNX/KVNX20110520_093548_V06.gz as snapshot G8P453WFS6ZAN1J9XC80
[icechun

## Read Data from Zarr Store

Read the converted data back from the Zarr store.

Create a readonly session to access the converted data.

In [6]:
session = repo.readonly_session("main")

Open the radar datatree using xarray.

In [7]:
dtree = xr.open_datatree(session.store, engine="zarr", consolidated=False, chunks={})

In [8]:
dtree

<xarray.DataTree>
Group: /
└── Group: /VCP-12
    │   Dimensions:        (vcp_time: 54)
    │   Coordinates:
    │     * vcp_time       (vcp_time) datetime64[ns] 432B 2011-05-20T09:03:33 ... 201...
    │   Data variables:
    │       altitude       (vcp_time) float64 432B dask.array<chunksize=(1,), meta=np.ndarray>
    │       latitude       (vcp_time) float32 216B dask.array<chunksize=(1,), meta=np.ndarray>
    │       longitude      (vcp_time) float32 216B dask.array<chunksize=(1,), meta=np.ndarray>
    │       volume_number  (vcp_time) float64 432B dask.array<chunksize=(1,), meta=np.ndarray>
    │   Attributes:
    │       Conventions:      None
    │       instrument_name:  KVNX
    │       version:          None
    │       title:            None
    │       institution:      None
    │       references:       None
    │       source:           None
    │       history:          None
    │       comment:          im/exported using xradar
    │       scan_name:        VCP-12
    ├── Group: /VCP-12/sweep_10
    │       Dimensions:            (vcp_time: 54, azimuth: 360, range: 924)
    │       Coordinates:
    │         * azimuth            (azimuth) float64 3kB 0.5 1.5 2.5 ... 357.5 358.5 359.5
    │           altitude           int64 8B ...
    │           longitude          float64 8B ...
    │           elevation          (azimuth) float64 3kB dask.array<chunksize=(360,), meta=np.ndarray>
    │           crs_wkt            int64 8B ...
    │         * range              (range) float32 4kB 2.125e+03 2.375e+03 ... 2.329e+05
    │           time               (azimuth) datetime64[ns] 3kB dask.array<chunksize=(360,), meta=np.ndarray>
    │           latitude           float64 8B ...
    │           y                  (azimuth, range) float64 3MB dask.array<chunksize=(90, 462), meta=np.ndarray>
    │           x                  (azimuth, range) float64 3MB dask.array<chunksize=(90, 462), meta=np.ndarray>
    │           z                  (azimuth, range) float64 3MB dask.array<chunksize=(90, 462), meta=np.ndarray>
    │       Data variables:
    │           WRADH              (vcp_time, azimuth, range) float32 72MB dask.array<chunksize=(1, 360, 924), meta=np.ndarray>
    │           DBZH               (vcp_time, azimuth, range) float32 72MB dask.array<chunksize=(1, 360, 924), meta=np.ndarray>
    │           ZDR                (vcp_time, azimuth, range) float32 72MB dask.array<chunksize=(1, 360, 924), meta=np.ndarray>
    │           RHOHV              (vcp_time, azimuth, range) float32 72MB dask.array<chunksize=(1, 360, 924), meta=np.ndarray>
    │           VRADH              (vcp_time, azimuth, range) float32 72MB dask.array<chunksize=(1, 360, 924), meta=np.ndarray>
    │           PHIDP              (vcp_time, azimuth, range) float32 72MB dask.array<chunksize=(1, 360, 924), meta=np.ndarray>
    │           sweep_fixed_angle  (vcp_time) float32 216B dask.array<chunksize=(1,), meta=np.ndarray>
    │           sweep_number       (vcp_time) float64 432B dask.array<chunksize=(1,), meta=np.ndarray>
    ├── Group: /VCP-12/georeferencing_correction
    │       Dimensions:    ()
    │       Coordinates:
    │           altitude   int64 8B ...
    │           latitude   float64 8B ...
    │           longitude  float64 8B ...
    ├── Group: /VCP-12/sweep_1
    │       Dimensions:            (vcp_time: 54, azimuth: 720, range: 1192)
    │       Coordinates:
    │         * range              (range) float32 5kB 2.125e+03 2.375e+03 ... 2.999e+05
    │         * azimuth            (azimuth) float64 6kB 0.25 0.75 1.25 ... 359.2 359.8
    │           latitude           float64 8B ...
    │           longitude          float64 8B ...
    │           elevation          (azimuth) float64 6kB dask.array<chunksize=(720,), meta=np.ndarray>
    │           altitude           int64 8B ...
    │           z                  (azimuth, range) float64 7MB dask.array<chunksize=(180, 298), meta=np.ndarray>
    │           crs_wkt            int64

The datatree contains VCP nodes organized by Volume Coverage Pattern.

In [9]:
list(dtree.children)

['VCP-12']

Navigate the radar datatree using path syntax like `VCP-12/sweep_0/DBZH`.

In [10]:
dtree["VCP-12/sweep_0/DBZH"]

<xarray.DataArray 'DBZH' (vcp_time: 54, azimuth: 720, range: 1832)> Size: 285MB
dask.array<open_dataset-DBZH, shape=(54, 720, 1832), dtype=float32, chunksize=(1, 720, 1832), chunktype=numpy.ndarray>
Coordinates:
  * vcp_time   (vcp_time) datetime64[ns] 432B 2011-05-20T09:03:33 ... 2011-05...
    elevation  (azimuth) float64 6kB dask.array<chunksize=(720,), meta=np.ndarray>
    crs_wkt    int64 8B ...
    longitude  float64 8B ...
  * range      (range) float32 7kB 2.125e+03 2.375e+03 ... 4.596e+05 4.599e+05
    altitude   int64 8B ...
    latitude   float64 8B ...
  * azimuth    (azimuth) float64 6kB 0.25 0.75 1.25 1.75 ... 358.8 359.2 359.8
    x          (azimuth, range) float64 11MB dask.array<chunksize=(180, 458), meta=np.ndarray>
    time       (azimuth) datetime64[ns] 6kB dask.array<chunksize=(720,), meta=np.ndarray>
    y          (azimuth, range) float64 11MB dask.array<chunksize=(180, 458), meta=np.ndarray>
    z          (azimuth, range) float64 11MB dask.array<chunksize=(180, 458), meta=np.ndarray>
Attributes:
    standard_name:  radar_equivalent_reflectivity_factor_h
    units:          dBZ
    long_name:      Equivalent reflectivity factor H

## Summary

This notebook demonstrated the core workflow for converting NEXRAD radar data to Zarr format:

1. **Discovery**: Used `list_nexrad_files()` to find radar files from AWS S3
2. **Conversion**: Applied `convert_files()` with sequential processing for simplicity  
3. **Access**: Read the hierarchical Zarr data back using xarray DataTree

The resulting Zarr store organizes radar data by Volume Coverage Pattern (VCP) and elevation sweeps, making it easy to access specific time periods and elevations for analysis.